# Lab 1.5.5: Training Diagnostics Lab

**Module:** 1.5 - Neural Network Fundamentals  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐ (Advanced Debugging Skills)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Identify common training problems from loss curves
- [ ] Diagnose learning rate issues (too high, too low)
- [ ] Detect and fix vanishing/exploding gradients
- [ ] Create a debugging checklist for neural networks
- [ ] Master the "overfit one batch" debugging technique

---

## 📚 Prerequisites

- Completed: Notebooks 01-04
- Knowledge of: Training loops, loss functions, activations

---

## 🌍 Real-World Context

**Debugging neural networks is an essential skill:**

Even experienced ML engineers spend significant time debugging training issues. The difference between a junior and senior ML engineer often comes down to debugging skills. This notebook teaches you to recognize symptoms and apply fixes quickly.

Famous quote from Andrej Karpathy: *"The most important skill for an ML engineer is the ability to debug models."*

---

## 🧒 ELI5: Why Neural Networks Fail

> **Imagine you're teaching a toddler to walk.**
>
> Things that can go wrong:
> - **Steps too big** (learning rate too high): They fall over constantly
> - **Steps too small** (learning rate too low): They barely make progress
> - **Can't feel their legs** (vanishing gradients): They don't know how to move
> - **Legs moving wildly** (exploding gradients): Completely out of control
> - **Only practicing in kitchen** (overfitting): Can't walk anywhere else
>
> As a "parent" (ML engineer), you need to watch for these signs and adjust the training accordingly!

---

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
import time
import sys
import os
from pathlib import Path

# Add scripts directory to path (robust approach)
notebook_dir = Path().resolve()
if notebook_dir.name == 'labs':
    scripts_dir = notebook_dir.parent / 'scripts'
else:
    scripts_dir = notebook_dir / 'scripts'
    if not scripts_dir.exists():
        scripts_dir = notebook_dir.parent / 'scripts'

if scripts_dir.exists():
    sys.path.insert(0, str(scripts_dir))

np.random.seed(42)
plt.style.use('default')
%matplotlib inline

print("Setup complete!")

In [ ]:
# Load MNIST for experiments
import gzip
import urllib.request

def load_mnist(path='../data'):
    os.makedirs(path, exist_ok=True)
    base_url = 'http://yann.lecun.com/exdb/mnist/'
    files = {
        'train_images': 'train-images-idx3-ubyte.gz',
        'train_labels': 'train-labels-idx1-ubyte.gz',
        'test_images': 't10k-images-idx3-ubyte.gz',
        'test_labels': 't10k-labels-idx1-ubyte.gz'
    }
    
    def download(filename):
        filepath = os.path.join(path, filename)
        if not os.path.exists(filepath):
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(base_url + filename, filepath)
        return filepath
    
    def load_images(fp):
        with gzip.open(fp, 'rb') as f:
            f.read(16)
            return np.frombuffer(f.read(), dtype=np.uint8).reshape(-1, 784).astype(np.float32) / 255.0
    
    def load_labels(fp):
        with gzip.open(fp, 'rb') as f:
            f.read(8)
            return np.frombuffer(f.read(), dtype=np.uint8)
    
    return (load_images(download(files['train_images'])),
            load_labels(download(files['train_labels'])),
            load_images(download(files['test_images'])),
            load_labels(download(files['test_labels'])))

X_train, y_train, X_test, y_test = load_mnist()
print(f"Loaded {len(X_train)} training samples")

In [ ]:
# Reusable MLP class for experiments
class DiagnosticMLP:
    """
    MLP with diagnostic capabilities.
    """
    
    def __init__(self, layer_sizes: List[int], activation: str = 'relu'):
        self.layers = []
        self.activation = activation
        self.gradient_history = []  # Track gradient magnitudes
        
        for i in range(len(layer_sizes) - 1):
            # Standard initialization
            W = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2.0 / layer_sizes[i])
            b = np.zeros(layer_sizes[i + 1])
            self.layers.append({'W': W, 'b': b, 'cache': {}, 'dW': None, 'db': None})
    
    def _activate(self, x):
        if self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
        elif self.activation == 'tanh':
            return np.tanh(x)
        return x
    
    def _activate_backward(self, x, dout):
        if self.activation == 'relu':
            return dout * (x > 0)
        elif self.activation == 'sigmoid':
            s = 1 / (1 + np.exp(-np.clip(x, -500, 500)))
            return dout * s * (1 - s)
        elif self.activation == 'tanh':
            return dout * (1 - np.tanh(x) ** 2)
        return dout
    
    def forward(self, X):
        out = X
        for i, layer in enumerate(self.layers[:-1]):
            layer['cache']['X'] = out
            out = out @ layer['W'] + layer['b']
            layer['cache']['Z'] = out
            out = self._activate(out)
        
        self.layers[-1]['cache']['X'] = out
        out = out @ self.layers[-1]['W'] + self.layers[-1]['b']
        
        out_shifted = out - np.max(out, axis=1, keepdims=True)
        exp_out = np.exp(out_shifted)
        self.probs = exp_out / np.sum(exp_out, axis=1, keepdims=True)
        return self.probs
    
    def backward(self, targets, lr):
        batch_size = len(targets)
        grad = self.probs.copy()
        grad[np.arange(batch_size), targets] -= 1
        
        layer_gradients = []
        
        for i in range(len(self.layers) - 1, -1, -1):
            layer = self.layers[i]
            X = layer['cache']['X']
            
            layer['dW'] = X.T @ grad / batch_size
            layer['db'] = np.mean(grad, axis=0)
            
            # Track gradient magnitude
            layer_gradients.append(np.abs(layer['dW']).mean())
            
            grad = grad @ layer['W'].T
            
            if i > 0:
                Z = self.layers[i - 1]['cache']['Z']
                grad = self._activate_backward(Z, grad)
            
            layer['W'] -= lr * layer['dW']
            layer['b'] -= lr * layer['db']
        
        self.gradient_history.append(layer_gradients[::-1])
    
    def predict(self, X):
        return np.argmax(self.forward(X), axis=1)

---

## Part 1: Problem 1 - Learning Rate Too High

### Symptoms:
- Loss oscillates wildly or explodes to infinity
- Training is unstable
- Accuracy fluctuates randomly

In [ ]:
def train_model(model, lr, epochs=20, batch_size=64, subset=5000):
    """Train model and return history."""
    history = {'loss': [], 'acc': []}
    X_sub, y_sub = X_train[:subset], y_train[:subset]
    
    for epoch in range(epochs):
        indices = np.random.permutation(len(X_sub))
        epoch_loss = 0
        n_batches = 0
        
        for start in range(0, len(X_sub), batch_size):
            batch_idx = indices[start:start + batch_size]
            X_batch = X_sub[batch_idx]
            y_batch = y_sub[batch_idx]
            
            probs = model.forward(X_batch)
            loss = -np.mean(np.log(probs[np.arange(len(y_batch)), y_batch] + 1e-10))
            
            # Clip loss for visualization (NaN becomes large number)
            if np.isnan(loss) or loss > 100:
                loss = 100
            
            model.backward(y_batch, lr)
            epoch_loss += loss
            n_batches += 1
        
        acc = np.mean(model.predict(X_test[:1000]) == y_test[:1000])
        history['loss'].append(epoch_loss / n_batches)
        history['acc'].append(acc)
    
    return history

In [ ]:
# Demonstrate learning rate too high
print("❌ Problem: Learning Rate Too High")
print("=" * 60)

np.random.seed(42)
model_high_lr = DiagnosticMLP([784, 256, 128, 10])
history_high_lr = train_model(model_high_lr, lr=10.0, epochs=20)  # LR=10 is way too high!

print(f"Final Loss: {history_high_lr['loss'][-1]:.4f}")
print(f"Final Accuracy: {history_high_lr['acc'][-1]:.2%}")
print("\n⚠️ Notice: Loss explodes or oscillates wildly!")

In [ ]:
# Visualize the problem
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history_high_lr['loss'], 'r-', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('❌ Loss with LR=10.0 (Too High)', fontsize=12)
axes[0].grid(True, alpha=0.3)

axes[1].plot(history_high_lr['acc'], 'r-', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('❌ Accuracy Fluctuates Randomly', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# The fix: Lower learning rate
print("\n✅ Fix: Lower the Learning Rate")
print("=" * 60)

np.random.seed(42)
model_good_lr = DiagnosticMLP([784, 256, 128, 10])
history_good_lr = train_model(model_good_lr, lr=0.1, epochs=20)  # LR=0.1 is reasonable

print(f"Final Loss: {history_good_lr['loss'][-1]:.4f}")
print(f"Final Accuracy: {history_good_lr['acc'][-1]:.2%}")

# Compare
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history_high_lr['loss'], 'r-', linewidth=2, alpha=0.5, label='LR=10.0 (bad)')
axes[0].plot(history_good_lr['loss'], 'g-', linewidth=2, label='LR=0.1 (good)')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history_high_lr['acc'], 'r-', linewidth=2, alpha=0.5, label='LR=10.0 (bad)')
axes[1].plot(history_good_lr['acc'], 'g-', linewidth=2, label='LR=0.1 (good)')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 2: Problem 2 - Learning Rate Too Low

### Symptoms:
- Loss decreases extremely slowly
- Training takes forever
- Model never reaches good performance

In [ ]:
print("❌ Problem: Learning Rate Too Low")
print("=" * 60)

np.random.seed(42)
model_low_lr = DiagnosticMLP([784, 256, 128, 10])
history_low_lr = train_model(model_low_lr, lr=0.0001, epochs=20)  # LR=0.0001 is too low

print(f"Final Loss: {history_low_lr['loss'][-1]:.4f}")
print(f"Final Accuracy: {history_low_lr['acc'][-1]:.2%}")
print("\n⚠️ Notice: Loss barely moves! Training is stuck.")

# Visualize
fig, ax = plt.subplots(figsize=(8, 4))

ax.plot(history_low_lr['loss'], 'b-', linewidth=2, label='LR=0.0001 (too low)')
ax.plot(history_good_lr['loss'], 'g-', linewidth=2, label='LR=0.1 (good)')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Learning Rate Too Low vs Good')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 3: Problem 3 - Vanishing Gradients (Deep Sigmoid Network)

### Symptoms:
- Gradients in early layers are nearly zero
- Early layers don't learn
- Loss decreases slowly even with good LR

In [ ]:
print("❌ Problem: Vanishing Gradients (Deep Sigmoid Network)")
print("=" * 60)

# Create a deep network with sigmoid activation
np.random.seed(42)
model_sigmoid = DiagnosticMLP([784, 256, 256, 256, 256, 128, 10], activation='sigmoid')
history_sigmoid = train_model(model_sigmoid, lr=0.1, epochs=20)

print(f"Final Accuracy: {history_sigmoid['acc'][-1]:.2%}")

# Show gradient magnitudes by layer
print("\nGradient magnitudes by layer (last epoch):")
last_grads = model_sigmoid.gradient_history[-1]
for i, g in enumerate(last_grads):
    bar = '█' * int(g * 1000)
    print(f"  Layer {i}: {g:.6f} {bar}")

print("\n⚠️ Notice: Gradients vanish in early layers!")

In [ ]:
# Visualize gradient flow
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gradient magnitudes over training
grads_over_time = np.array(model_sigmoid.gradient_history)
for layer_idx in range(grads_over_time.shape[1]):
    axes[0].plot(grads_over_time[:, layer_idx], label=f'Layer {layer_idx}')

axes[0].set_xlabel('Training Step (batch)')
axes[0].set_ylabel('Gradient Magnitude')
axes[0].set_title('Gradient Magnitudes (Sigmoid Network)')
axes[0].legend(fontsize=8)
axes[0].set_yscale('log')
axes[0].grid(True, alpha=0.3)

# Compare final gradients
layer_names = [f'Layer {i}' for i in range(len(last_grads))]
axes[1].bar(layer_names, last_grads, color=['red' if g < 0.001 else 'green' for g in last_grads])
axes[1].set_ylabel('Gradient Magnitude')
axes[1].set_title('Final Gradient by Layer (Red = Vanishing)')
axes[1].axhline(0.001, color='red', linestyle='--', label='Vanishing threshold')

plt.tight_layout()
plt.show()

In [ ]:
# The fix: Use ReLU activation
print("\n✅ Fix: Use ReLU Activation")
print("=" * 60)

np.random.seed(42)
model_relu = DiagnosticMLP([784, 256, 256, 256, 256, 128, 10], activation='relu')
history_relu = train_model(model_relu, lr=0.1, epochs=20)

print(f"Final Accuracy (ReLU): {history_relu['acc'][-1]:.2%}")

print("\nGradient magnitudes by layer (ReLU):")
last_grads_relu = model_relu.gradient_history[-1]
for i, g in enumerate(last_grads_relu):
    bar = '█' * int(g * 100)
    print(f"  Layer {i}: {g:.6f} {bar}")

# Compare
fig, ax = plt.subplots(figsize=(8, 4))
x = np.arange(len(last_grads))
width = 0.35

ax.bar(x - width/2, last_grads, width, label='Sigmoid', color='red', alpha=0.7)
ax.bar(x + width/2, last_grads_relu, width, label='ReLU', color='green', alpha=0.7)
ax.set_xlabel('Layer')
ax.set_ylabel('Gradient Magnitude')
ax.set_title('Gradient Comparison: Sigmoid vs ReLU')
ax.legend()
ax.set_yscale('log')

plt.tight_layout()
plt.show()

---

## Part 4: The "Overfit One Batch" Test

### The Most Important Debugging Technique

Before training on full data, always test if your model can memorize a single batch. If it can't, there's a bug in your implementation!

In [ ]:
print("🔬 The 'Overfit One Batch' Test")
print("=" * 60)

# Take a single small batch
X_batch = X_train[:32]
y_batch = y_train[:32]

print(f"Batch size: {len(X_batch)}")
print("\nIf your model works, it should perfectly memorize this batch.")
print("Loss should go to ~0, accuracy should go to 100%.")

# Train on just this batch
np.random.seed(42)
model_overfit = DiagnosticMLP([784, 256, 128, 10])

losses = []
accs = []

for i in range(200):  # Many iterations on same batch
    probs = model_overfit.forward(X_batch)
    loss = -np.mean(np.log(probs[np.arange(len(y_batch)), y_batch] + 1e-10))
    model_overfit.backward(y_batch, lr=0.1)
    
    acc = np.mean(model_overfit.predict(X_batch) == y_batch)
    losses.append(loss)
    accs.append(acc)

print(f"\nAfter 200 iterations:")
print(f"  Loss: {losses[-1]:.6f} (should be ~0)")
print(f"  Accuracy: {accs[-1]:.2%} (should be 100%)")

if accs[-1] > 0.99:
    print("\n✅ Model can overfit one batch - implementation is correct!")
else:
    print("\n❌ Model CAN'T overfit one batch - there's a bug!")

In [ ]:
# Visualize the overfit test
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(losses, 'b-', linewidth=2)
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Should Approach 0')
axes[0].grid(True, alpha=0.3)

axes[1].plot(accs, 'g-', linewidth=2)
axes[1].axhline(1.0, color='red', linestyle='--', label='100% target')
axes[1].set_xlabel('Iteration')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy Should Reach 100%')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 5: The Complete Debugging Checklist

Here's your go-to checklist when training fails:

In [ ]:
print("\n" + "=" * 80)
print("                    NEURAL NETWORK DEBUGGING CHECKLIST")
print("=" * 80)

checklist = """
1. ✅ CAN YOU OVERFIT ONE BATCH?
   → Train on a single batch for 100+ iterations
   → Loss should → 0, accuracy should → 100%
   → If not: Bug in forward/backward pass

2. ✅ IS YOUR DATA CORRECT?
   → Visualize some samples with labels
   → Check for NaN/Inf values
   → Verify normalization (mean~0, std~1)

3. ✅ IS LOSS DECREASING AT ALL?
   → If no: Learning rate too low OR gradient issue
   → Try increasing LR by 10x
   → Check gradients aren't zero

4. ✅ IS LOSS EXPLODING OR OSCILLATING?
   → Learning rate too high
   → Try decreasing LR by 10x
   → Add gradient clipping

5. ✅ IS LOSS NaN OR Inf?
   → Numerical instability
   → Check for division by zero
   → Add epsilon to denominators
   → Clip activations/logits

6. ✅ ARE GRADIENTS VANISHING?
   → Check gradient magnitudes per layer
   → Switch from Sigmoid/Tanh to ReLU
   → Use proper initialization (He for ReLU)
   → Add residual connections

7. ✅ IS VALIDATION LOSS INCREASING WHILE TRAINING LOSS DECREASES?
   → Overfitting!
   → Add regularization (L2, Dropout)
   → Get more data
   → Reduce model size

8. ✅ IS ACCURACY STUCK AT RANDOM CHANCE?
   → Labels might be shuffled incorrectly
   → Output layer issue
   → Check class imbalance
"""

print(checklist)

---

## Part 6: Symptoms → Diagnosis → Fix Reference

In [ ]:
print("\n" + "=" * 100)
print("                              SYMPTOM → DIAGNOSIS → FIX")
print("=" * 100)

print("""
┌─────────────────────────────────┬─────────────────────────────┬─────────────────────────────────┐
│ SYMPTOM                         │ DIAGNOSIS                   │ FIX                             │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Loss explodes to NaN/Inf        │ LR too high OR              │ Decrease LR by 10x              │
│                                 │ Numerical instability       │ Add gradient clipping           │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Loss barely moves               │ LR too low OR               │ Increase LR by 10x              │
│                                 │ Vanishing gradients         │ Use ReLU, check initialization  │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Loss oscillates wildly          │ LR too high OR              │ Decrease LR                     │
│                                 │ Batch size too small        │ Increase batch size             │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Can't overfit single batch      │ Bug in implementation       │ Debug forward/backward pass     │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Train loss ↓, Val loss ↑        │ Overfitting                 │ Add regularization, get data    │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Both losses high, not moving    │ Underfitting                │ Bigger model, train longer      │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Accuracy stuck at random        │ Labels issue OR             │ Check data labels               │
│                                 │ Output layer bug            │ Verify softmax/loss             │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ Early layer gradients = 0       │ Vanishing gradients         │ Use ReLU, He init, ResNets      │
├─────────────────────────────────┼─────────────────────────────┼─────────────────────────────────┤
│ All predictions are same class  │ Class imbalance OR          │ Balance data, check loss        │
│                                 │ Bug in output layer         │                                 │
└─────────────────────────────────┴─────────────────────────────┴─────────────────────────────────┘
""")

---

## ✋ Try It Yourself

### Exercise: Diagnose These Training Curves

Look at each curve and identify the problem. What would you fix?

In [ ]:
# Generate example problematic curves
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Problem 1: Exploding loss
x = np.arange(20)
y1 = np.exp(x * 0.3) + np.random.randn(20) * 10
axes[0, 0].plot(y1, 'r-', linewidth=2)
axes[0, 0].set_title('Problem A: ?????', fontsize=12)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].grid(True, alpha=0.3)

# Problem 2: Flat loss
y2 = 2.3 + np.random.randn(20) * 0.01
axes[0, 1].plot(y2, 'b-', linewidth=2)
axes[0, 1].set_title('Problem B: ?????', fontsize=12)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_ylim(0, 5)
axes[0, 1].grid(True, alpha=0.3)

# Problem 3: Overfitting
y3_train = 2.3 - np.log(x + 1) * 0.8 + np.random.randn(20) * 0.05
y3_val = 2.3 - np.log(x + 1) * 0.3 + x * 0.05 + np.random.randn(20) * 0.05
axes[1, 0].plot(y3_train, 'b-', linewidth=2, label='Train')
axes[1, 0].plot(y3_val, 'r-', linewidth=2, label='Validation')
axes[1, 0].set_title('Problem C: ?????', fontsize=12)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Problem 4: Oscillating
y4 = 1.5 + np.sin(x * 0.5) * 0.5 + np.random.randn(20) * 0.3
axes[1, 1].plot(y4, 'g-', linewidth=2)
axes[1, 1].set_title('Problem D: ?????', fontsize=12)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🤔 Can you identify each problem?")
print("   A: _____________")
print("   B: _____________")
print("   C: _____________")
print("   D: _____________")

<details>
<summary>💡 Answers</summary>

- **A**: Learning rate too high (loss exploding)
- **B**: Learning rate too low (loss stuck)
- **C**: Overfitting (train decreases, val increases)
- **D**: Learning rate too high (oscillating)
</details>

---

## 🎉 Checkpoint

You've learned:

- ✅ How to identify LR too high/low from loss curves
- ✅ How vanishing gradients manifest and how to fix them
- ✅ The "overfit one batch" debugging technique
- ✅ A complete debugging checklist

---

## 📖 Further Reading

- [A Recipe for Training Neural Networks (Karpathy)](http://karpathy.github.io/2019/04/25/recipe/)
- [Troubleshooting Deep Neural Networks (Josh Tobin)](https://josh-tobin.com/assets/pdf/troubleshooting-deep-neural-networks-01-19.pdf)

---

## 🧹 Cleanup

In [ ]:
import gc
gc.collect()

print("✅ Cleanup complete!")
print("\n🎯 Next: Proceed to notebook 06-gpu-acceleration.ipynb")